In [1]:
'''
This assumes the highest z score for Default Network (DN) is the central posterior network.
'''
%matplotlib inline
from pathlib import *
from io import StringIO
import numpy as np
import pandas as pd
import nibabel as nib
import mmimproc as ip
from mmimproc.io.mixed import df2h5
from mmimproc.utils import run_subprocess
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
pd.set_option('display.max_colwidth', 90)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
  
def append2fn(fn, newstr):
    """Appends new string to end of file name and before file extentions.
    """
    return Path(fn).stem + newstr + ''.join(Path(fn).suffixes)

def fslcluster2list(cluster_output):
    return [line.split('\t') for line in StringIO(cluster_output.decode('UTF-8')).read().split('\n')]
    
def fslcluster2DF(fname, thresh, *argv):
    if not Path(fname).is_file():
        raise ValueError('Cannot find stats file '+str(fname))   
    cluster_output = run_subprocess(' '.join(['cluster', '-i', fname, '-t', str(thresh), *argv]))
    cluster_data = fslcluster2list(cluster_output[0])
    cluster_df = pd.DataFrame(cluster_data[1:], columns=cluster_data[0], dtype=np.float)
    cluster_df.replace('', np.nan, inplace=True)
    cluster_df.dropna(how='all', inplace=True)
    return cluster_df

ssvolnum = 10    # integer volume number where steady state is acheived 

# set up file naming
datadir = ip.fs_local   # enter pathlib or string for BIDS root data directory
proj = 'toddandclark'   # enter project name
subj = 'connect_dn'  # enter subject name (todo: subject list option for for loop)
sess = 1
mod = 'rest_1'
stats_dir = 'stats'
niiname = 'Step0Orig.nii.gz'  # name of raw resting nifti file to process (todo: for loop if needed)
statsfile = 'zstat1.nii.gz'
seedfile = 'shadowreg_centralsignal.nii.gz'
resultsh5 = 'tcj_results1.h5'

namedict  = {'datadir': datadir, 'proj': proj, 'subj': subj, 'sess': 'session_{0}'.format(sess), 'modality': mod, 'niiname': niiname,
              'maskname': append2fn(niiname, '_mask'), 'snrmaskname': append2fn(niiname, '_snrmask'), 'statsfile': statsfile,
              'seedfile': seedfile, 'resultsname': resultsh5}

cluster_df =  fslcluster2DF('{datadir}/{proj}/{subj}/{sess}/stats/{statsfile}'.format(**namedict), 9.5)
cluster_df.loc[cluster_df['MAX'].idxmax(axis=1), 'location'] = 'central'
cluster_df.loc[cluster_df.loc[:,'MAX X (vox)']  <  cluster_df.loc[cluster_df.loc[:,'location'] == 'central']['MAX X (vox)'][0],'location']  = 'left'
cluster_df.loc[cluster_df.loc[:,'MAX X (vox)']  >  cluster_df.loc[cluster_df.loc[:,'location'] == 'central']['MAX X (vox)'][0],'location']  = 'right'
cluster_df.set_index('location', inplace=True )
coordcols = ['MAX X (vox)',	'MAX Y (vox)',	'MAX Z (vox)']
cluster_df.loc[cluster_df.index == 'central', 'dist2central'] = 0.0
cluster_df.loc[cluster_df.index == 'right', 'dist2central'] = \
    np.linalg.norm(cluster_df.loc[cluster_df.index == 'central',:][coordcols].values - cluster_df.loc[cluster_df.index == 'right',:][coordcols].values)
cluster_df.loc[cluster_df.index == 'left', 'dist2central'] = \
    np.linalg.norm(cluster_df.loc[cluster_df.index == 'central',:][coordcols].values - cluster_df.loc[cluster_df.index == 'left',:][coordcols].values)
cluster_df
'''
make mask containing both cylinders.
r = 1 is a line.
r = 2 is cylinder of diameter 3 voxels
r = 3 is cylinder of diameter 5 voxels
r = 4 is cylinder of diameter 7 voxels
lets stop there.
set index for each segment of cylinder to unique label number for stats extraction left: 1 to 20. right: 30-50
make break points in y and z dividing up (segmenting) line along x. increment by one voxel at each break point in the direction of left or right point.
left: x is decreasing. y is -1 and z is also -1. distance is 14 so at vox count of 7 along x line both y and z should decrease by 1 and hold. x stops at 14 (left max x).
right: x is increasing. y is +1 and z is also +1. distance is 11 so vox count of 5 (round down) along x line both y and z should increase by 1 and hold.
x stops at 39 (right max x).
make x radius fill function has r value and coord to surround x in y and z plane.
make a table of median zscore values for each point along the line.
explore directly sampling zscore file by coordinate.
graph median zscores center to left and center to right
'''

,Cluster Index,Voxels,MAX,MAX X (vox),MAX Y (vox),MAX Z (vox),COG X (vox),COG Y (vox),COG Z (vox),dist2central
location,,,,,,,,,,
central,3,129.0,23.5,28.0,19.0,24.0,27.8,20.4,23.4,0.000000
left,2,6.0,10.3,14.0,18.0,23.0,14.3,17.7,22.5,14.071247
right,1,3.0,10.0,39.0,20.0,25.0,39.3,20.0,25.3,11.090537


In [6]:
cluster_df.index == 'central'

array([ True, False, False])

In [28]:
centralcoord = cluster_df.loc[cluster_df.index == 'central',:][coordcols].values
leftcoord = cluster_df.loc[cluster_df.index == 'left',:][coordcols].values
rightcoord = cluster_df.loc[cluster_df.index == 'right',:][coordcols].values

np.sqrt(np.sum(np.square(rightcoord - centralcoord)))

11.090536506409418

In [25]:
np.linalg.norm(centralcoord - leftcoord)

14.071247279470288

In [29]:
np.linalg.norm(cluster_df.loc[cluster_df.index == 'central',:][coordcols].values - cluster_df.loc[cluster_df.index == 'right',:][coordcols].values)

11.090536506409418

KeyError: False

pandas.core.series.Series

,Cluster Index,Voxels,MAX,MAX X (vox),MAX Y (vox),MAX Z (vox),COG X (vox),COG Y (vox),COG Z (vox)
0,3,129.0,23.5,28.0,19.0,24.0,27.8,20.4,23.4
1,2,6.0,10.3,14.0,18.0,23.0,14.3,17.7,22.5
2,1,3.0,10.0,39.0,20.0,25.0,39.3,20.0,25.3
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Cluster Index,Voxels,MAX,MAX X (vox),MAX Y (vox),MAX Z (vox),COG X (vox),COG Y (vox),COG Z (vox)
0,3,129.0,23.5,28.0,19.0,24.0,27.8,20.4,23.4
1,2,6.0,10.3,14.0,18.0,23.0,14.3,17.7,22.5
2,1,3.0,10.0,39.0,20.0,25.0,39.3,20.0,25.3
